In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 11, 2),
 datetime.date(2020, 11, 1),
 datetime.date(2020, 10, 31))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201102.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201101.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201031.csv')

In [4]:
wait = True
if wait:
    while True:
        response = requests.get(url_tday)
        if response.status_code == 200:
            print('Presenti dati odierni')
            date_last = date_tday
            csv = response.content
            df_raw_r_1 = pd.read_csv(io.StringIO(csv.decode('utf-8')))
            df_raw_r_0 = pd.read_csv(url_yday)
            break
        else:
            print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)
else:
    print('Dati di ieri')
    date_last = date_yday
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)

Presenti dati odierni


In [5]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-02T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3108,196,3304,32791,...,1681,2003,34153,4415,34275,40388,74663,1050421,652762,NaN
1,2020-11-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,167,12,179,1833,...,92,122,1306,179,3080,417,3497,42151,26356,NaN
2,2020-11-02T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4406,435,4841,89577,...,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,NaN
3,2020-11-02T17:00:00,ITA,5,Veneto,45.434905,12.338452,873,136,1009,31924,...,1519,1544,25437,2427,25103,35694,60797,2350517,912972,NaN
4,2020-11-02T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,179,37,216,5337,...,177,218,5506,403,10146,1316,11462,545374,231668,NaN
5,2020-11-02T17:00:00,ITA,7,Liguria,44.411493,8.932699,1133,57,1190,8104,...,580,599,18859,1790,21510,8433,29943,450566,232521,NaN
6,2020-11-02T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1382,138,1520,24972,...,1575,1652,28093,4664,43112,16137,59249,1612029,865327,NaN
7,2020-11-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,1217,182,1399,30336,...,1761,2009,15529,1387,39186,9465,48651,1118732,743951,effettuati 15426 tamponi rapidi antigenici di ...
8,2020-11-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,308,46,354,6972,...,116,193,3674,140,3290,7850,11140,302745,175793,NaN
9,2020-11-02T17:00:00,ITA,11,Marche,43.616760,13.518875,334,50,384,6658,...,319,373,7110,1025,15000,177,15177,325563,190758,NaN


In [6]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-01T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2844,179,3023,31391,...,1590,2024,33852,4394,33810,38850,72660,1039308,643829,NaN
1,2020-11-01T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,161,12,173,1747,...,107,135,1281,174,2972,403,3375,41665,26164,NaN
2,2020-11-01T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4246,418,4664,85411,...,8064,8607,96687,17589,162163,42188,204351,2984274,1856584,NaN
3,2020-11-01T17:00:00,ITA,5,Veneto,45.434905,12.338452,832,132,964,30450,...,2235,2300,25421,2418,25008,34245,59253,2342686,910040,NaN
4,2020-11-01T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,180,38,218,5158,...,244,403,5470,398,9981,1263,11244,542817,230864,NaN
5,2020-11-01T17:00:00,ITA,7,Liguria,44.411493,8.932699,1080,63,1143,7571,...,283,736,18849,1781,21066,8278,29344,447720,231090,NaN
6,2020-11-01T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1267,132,1399,23518,...,1704,1758,28033,4647,42249,15348,57597,1601730,858385,In seguito a verifica sui dati sono stati elim...
7,2020-11-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,1106,173,1279,28695,...,2130,2379,15305,1363,37515,9127,46642,1106715,736419,NaN
8,2020-11-01T17:00:00,ITA,10,Umbria,43.106758,12.388247,295,46,341,6869,...,656,768,3602,135,3247,7700,10947,302117,175572,NaN
9,2020-11-01T17:00:00,ITA,11,Marche,43.616760,13.518875,327,42,369,6354,...,618,683,7060,1021,14654,150,14804,323790,189574,NaN


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-02T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3108,196,3304,32791,...,1681,2003,34153,4415,34275,40388,74663,1050421,652762,NaN
1,2020-11-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,167,12,179,1833,...,92,122,1306,179,3080,417,3497,42151,26356,NaN
2,2020-11-02T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4406,435,4841,89577,...,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,NaN
3,2020-11-02T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,471,34,505,7724,...,498,624,9896,763,14768,4120,18888,531143,237796,NaN
4,2020-11-02T17:00:00,ITA,5,Veneto,45.434905,12.338452,873,136,1009,31924,...,1519,1544,25437,2427,25103,35694,60797,2350517,912972,NaN
5,2020-11-02T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,179,37,216,5337,...,177,218,5506,403,10146,1316,11462,545374,231668,NaN
6,2020-11-02T17:00:00,ITA,7,Liguria,44.411493,8.932699,1133,57,1190,8104,...,580,599,18859,1790,21510,8433,29943,450566,232521,NaN
7,2020-11-02T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1382,138,1520,24972,...,1575,1652,28093,4664,43112,16137,59249,1612029,865327,NaN
8,2020-11-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,1217,182,1399,30336,...,1761,2009,15529,1387,39186,9465,48651,1118732,743951,effettuati 15426 tamponi rapidi antigenici di ...
9,2020-11-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,308,46,354,6972,...,116,193,3674,140,3290,7850,11140,302745,175793,NaN


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-11-01T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2844,179,3023,...,1590,2024,33852,4394,33810,38850,72660,1039308,643829,NaN
1,17,2020-11-01T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,161,12,173,...,107,135,1281,174,2972,403,3375,41665,26164,NaN
2,8,2020-11-01T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4246,418,4664,...,8064,8607,96687,17589,162163,42188,204351,2984274,1856584,NaN
3,19,2020-11-01T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,425,26,451,...,706,744,9777,756,14227,4037,18264,527927,236473,NaN
4,18,2020-11-01T17:00:00,ITA,5,Veneto,45.434905,12.338452,832,132,964,...,2235,2300,25421,2418,25008,34245,59253,2342686,910040,NaN
5,5,2020-11-01T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,180,38,218,...,244,403,5470,398,9981,1263,11244,542817,230864,NaN
6,7,2020-11-01T17:00:00,ITA,7,Liguria,44.411493,8.932699,1080,63,1143,...,283,736,18849,1781,21066,8278,29344,447720,231090,NaN
7,4,2020-11-01T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1267,132,1399,...,1704,1758,28033,4647,42249,15348,57597,1601730,858385,In seguito a verifica sui dati sono stati elim...
8,15,2020-11-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,1106,173,1279,...,2130,2379,15305,1363,37515,9127,46642,1106715,736419,NaN
9,16,2020-11-01T17:00:00,ITA,10,Umbria,43.106758,12.388247,295,46,341,...,656,768,3602,135,3247,7700,10947,302117,175572,NaN


In [9]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [10]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [11]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-02T17:00:00,1,Piemonte,3108,196,3304,32791,36095,1681,2003,34153,4415,34275,40388,74663,1050421,652762,4341375
1,2020-11-02T17:00:00,2,Valle d'Aosta,167,12,179,1833,2012,92,122,1306,179,3080,417,3497,42151,26356,125501
2,2020-11-02T17:00:00,3,Lombardia,4406,435,4841,89577,94418,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,10103969
3,2020-11-02T17:00:00,4,Trentino-Alto Adige,471,34,505,7724,8229,498,624,9896,763,14768,4120,18888,531143,237796,1074819
4,2020-11-02T17:00:00,5,Veneto,873,136,1009,31924,32933,1519,1544,25437,2427,25103,35694,60797,2350517,912972,4907704
5,2020-11-02T17:00:00,6,Friuli Venezia Giulia,179,37,216,5337,5553,177,218,5506,403,10146,1316,11462,545374,231668,1211357
6,2020-11-02T17:00:00,7,Liguria,1133,57,1190,8104,9294,580,599,18859,1790,21510,8433,29943,450566,232521,1543127
7,2020-11-02T17:00:00,8,Emilia-Romagna,1382,138,1520,24972,26492,1575,1652,28093,4664,43112,16137,59249,1612029,865327,4467118
8,2020-11-02T17:00:00,9,Toscana,1217,182,1399,30336,31735,1761,2009,15529,1387,39186,9465,48651,1118732,743951,3722729
9,2020-11-02T17:00:00,10,Umbria,308,46,354,6972,7326,116,193,3674,140,3290,7850,11140,302745,175793,880285


In [12]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-02T17:00:00,1,Piemonte,3108,196,3304,32791,36095,1681,2003,34153,4415,34275,40388,74663,1050421,652762,4341375,367
1,2020-11-02T17:00:00,2,Valle d'Aosta,167,12,179,1833,2012,92,122,1306,179,3080,417,3497,42151,26356,125501,20
2,2020-11-02T17:00:00,3,Lombardia,4406,435,4841,89577,94418,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,10103969,983
3,2020-11-02T17:00:00,4,Trentino-Alto Adige,471,34,505,7724,8229,498,624,9896,763,14768,4120,18888,531143,237796,1074819,106
4,2020-11-02T17:00:00,5,Veneto,873,136,1009,31924,32933,1519,1544,25437,2427,25103,35694,60797,2350517,912972,4907704,825
5,2020-11-02T17:00:00,6,Friuli Venezia Giulia,179,37,216,5337,5553,177,218,5506,403,10146,1316,11462,545374,231668,1211357,175
6,2020-11-02T17:00:00,7,Liguria,1133,57,1190,8104,9294,580,599,18859,1790,21510,8433,29943,450566,232521,1543127,209
7,2020-11-02T17:00:00,8,Emilia-Romagna,1382,138,1520,24972,26492,1575,1652,28093,4664,43112,16137,59249,1612029,865327,4467118,516
8,2020-11-02T17:00:00,9,Toscana,1217,182,1399,30336,31735,1761,2009,15529,1387,39186,9465,48651,1118732,743951,3722729,415
9,2020-11-02T17:00:00,10,Umbria,308,46,354,6972,7326,116,193,3674,140,3290,7850,11140,302745,175793,880285,70


In [13]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [14]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-02T17:00:00,3,Lombardia,4406,435,4841,89577,94418,4343,5278,...,44.25,4166,-3329,24087,14954,46,889,21.91,0.05224,2.07
1,2020-11-02T17:00:00,15,Campania,1486,171,1657,47113,48770,1592,2861,...,40.05,1521,-999,15632,10648,24,1245,18.30,0.04945,1.08
2,2020-11-02T17:00:00,9,Toscana,1217,182,1399,30336,31735,1761,2009,...,43.86,1641,-370,12017,7532,24,224,16.72,0.05397,1.31
3,2020-11-02T17:00:00,1,Piemonte,3108,196,3304,32791,36095,1681,2003,...,53.41,1400,-21,11113,8933,21,301,18.02,0.04614,1.72
4,2020-11-02T17:00:00,12,Lazio,2032,185,2217,35566,37783,1677,1859,...,24.77,1700,-492,19574,14574,23,159,9.50,0.03169,0.86
5,2020-11-02T17:00:00,8,Emilia-Romagna,1382,138,1520,24972,26492,1575,1652,...,26.74,1454,-106,10299,6942,17,60,16.04,0.03698,1.33
6,2020-11-02T17:00:00,5,Veneto,873,136,1009,31924,32933,1519,1544,...,16.48,1474,-756,7831,2932,9,16,19.72,0.03146,1.24
7,2020-11-02T17:00:00,19,Sicilia,1025,142,1167,14897,16064,740,1024,...,26.39,704,-71,8034,4790,18,266,12.75,0.02061,0.48
8,2020-11-02T17:00:00,16,Puglia,705,88,793,11776,12569,537,626,...,24.04,508,-54,4060,1737,9,80,15.42,0.01562,0.50
9,2020-11-02T17:00:00,4,Trentino-Alto Adige,471,34,505,7724,8229,498,624,...,32.08,444,-120,3216,1323,7,119,19.40,0.05806,1.76


In [15]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)